In [ ]:
!pip install transformers

In [ ]:
!pip install rouge

In [ ]:
EVAL = False # set if running in eval only mode

In [ ]:
!pip install -q tensorboard

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/'My Drive'/summ_data/cnn/

checkpoints  icsiruns  logs  processing  result  result2  tboard  tensors


In [ ]:
import pandas as pd
import numpy as np
import os
import sys
import tensorflow as tf
import torch
import datetime, time
import io
import re
from csv import reader
import matplotlib.pyplot as plt
import traceback
import random
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib.ticker import PercentFormatter
from torch.utils.tensorboard import SummaryWriter

import glob, os
import torch.nn as nn

from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from torch.utils.data import TensorDataset
from transformers import BertModel, AdamW, BertConfig,BertTokenizer,BertPreTrainedModel
from torch.nn.init import xavier_uniform_
from sklearn import preprocessing

import tensorflow as tf
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from rouge import Rouge 
import logging
from collections import OrderedDict, defaultdict


## Be very careful with block below (ensure that it is deleting the results directory and nothing else in your drive!)

In [ ]:
BASE_DIR = "/content/drive/My Drive/summ_data/cnnv2/"
INPUT_DIR = "/content/drive/My Drive/summ_data/cnn/"+"processing/"
DAT_DIR = BASE_DIR+"tensors/"
RESULT_DIR = BASE_DIR+"result2/"
TBOARD_LOG_DIR = BASE_DIR+"tboard/"
LOG_DIR = BASE_DIR+"logs/"

CP_DIR = "/content/drive/My Drive/summ_data/cnn/"+"checkpoints/"



In [ ]:
for dir in (DAT_DIR, RESULT_DIR, TBOARD_LOG_DIR, CP_DIR, LOG_DIR):
    if not os.path.exists(dir):
        os.makedirs(dir)

if not EVAL:
    for dir in (RESULT_DIR, TBOARD_LOG_DIR):
        for f in glob.glob(dir+"*"):
            os.remove(f)

else:
    for dir in (RESULT_DIR,):
        for f in glob.glob(dir+"*"):
            print(f)
            os.remove(f)


In [ ]:
log_file = LOG_DIR+datetime.datetime.now().strftime("%Y%m%d.%H.%M")+".txt"
logging.basicConfig(filename=log_file, level=logging.DEBUG)

logger = logging.getLogger()
logger.addHandler(logging.StreamHandler(sys.stdout))

In [ ]:
def save_cp(model_state, optimizer_state, scheduler, epoch):
    cp = {'model_state': model_state, 'optimizer_state':optimizer_state, 'scheduler': scheduler}
    filename = datetime.datetime.now().strftime("%Y%m%d.%H.")+str(epoch)+".pth.tar"
    torch.save(cp, CP_DIR+filename )

def load_cp(filename):
    cp = torch.load(filename)
    return cp['model_state'], cp['optimizer_state'], cp['scheduler']

logger.debug("logging initializer")
print(log_file)

logging initializer
/content/drive/My Drive/summ_data/cnnv2/logs/20201129.12.18.txt


## Create model inputs

In [ ]:
files = glob.glob(INPUT_DIR+'*')
print(INPUT_DIR)
for f in files:
    pass


/content/drive/My Drive/summ_data/cnn/processing/


In [ ]:
train_d, test_d, val_d = defaultdict(list), defaultdict(list), defaultdict(list)
src_maps = dict()
for d_type, d_dict in zip(('train', 'valid', 'test'), (train_d, test_d, val_d)):
    row_idx = 0

    for file_idx, inp_file in enumerate(glob.glob(INPUT_DIR+"*"+d_type+"*")):
        file_d = torch.load(inp_file)
        #we have ['src', 'labels', 'segs', 'clss', 'src_txt', 'tgt_txt']
        #we need ['idx', 'src', 'labels', 'segs', 'clss', 'attn', 'mask_cls']
        for rownum, dat in enumerate(file_d):
            d_dict['idx'].append(rownum)
            for k in ('src', 'labels', 'segs', 'clss'):
                dat[k].extend([0 for _ in range(512-len(dat[k]))])
                d_dict[k].append(dat[k])

            #attn
            attn= [1 if x!=0 else 0 for x in dat['src'] ]
            d_dict['attn'].append(attn)

            # mask_cls
            mask_cls = [1 if x!=0 else 0 for x in dat['clss'] ]
            if dat['clss'][0] == 0:
                mask_cls[0] = 1
            d_dict['mask_cls'].append(mask_cls)

            #save src and target text
            src_maps[d_type+"_src_"+str(rownum)] = dat["src_txt"]
            src_maps[d_type+"_tgt_"+str(rownum)] = dat["tgt_txt"]

            row_idx += 1
#        if (file_idx%75) == 0:
#            break
    for k, v in d_dict.items():
        d_dict[k] = torch.LongTensor(v)

train_dataset = TensorDataset(train_d["idx"], train_d["src"],train_d["labels"], train_d["segs"], 
                              train_d["clss"], train_d["attn"], train_d["mask_cls"])
val_dataset = TensorDataset(val_d["idx"], val_d["src"], val_d["labels"], val_d["segs"], 
                              val_d["clss"], val_d["attn"], val_d["mask_cls"])
test_dataset = TensorDataset(test_d["idx"], test_d["src"], test_d["labels"], test_d["segs"], 
                              test_d["clss"], test_d["attn"], test_d["mask_cls"])


## Encoder classes

In [ ]:
class BertForSummarization(BertPreTrainedModel):
    def __init__(self, config):
        super().__init__(config)
        self.model = BertModel(config)
        # Initialize model weights (inherited function).
        self.init_weights()


    def forward(self, x, segs, mask):
        # the below returns a tuple. First element in the tuple is last hidden state. Second element in tuple is pooler output
        self.result = self.model(input_ids=x, attention_mask =mask, token_type_ids=segs)
        top_vec = self.result[0]
        return top_vec


class Classifier(nn.Module):
    def __init__(self, hidden_size):
        super(Classifier, self).__init__()
        self.linear1 = nn.Linear(hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, mask_cls):
#        h = self.linear2(self.tanh(self.linear1(x)))
        h = self.linear1(x)
        h = h.squeeze(-1)
        sent_scores = self.sigmoid(h) * mask_cls.float()
        return sent_scores, x, h



class Summarizer(nn.Module):
    def __init__(self, args=None, num_hidden = 768, load_pretrained_bert = True, bert_config = None):
        super(Summarizer, self).__init__()
        self.args = args
        self.bert = BertForSummarization.from_pretrained(
            "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = False, # Whether the model returns all hidden-states.
        )
#        if (args.encoder == 'classifier'):
        self.encoder = Classifier(num_hidden)
        logger.debug("dropout = "+str(nn.Dropout(self.bert.config.hidden_dropout_prob)))
        self.dropout = nn.Dropout(self.bert.config.hidden_dropout_prob)
        for p in self.encoder.parameters():
            if p.dim() > 1:
                xavier_uniform_(p)

    def load_cp(self, state):
        self.load_state_dict(state, strict=True)

    def forward(self, x, segs, clss, mask, mask_cls, sentence_range=None):
        top_vec = self.bert(x, segs, mask)
#        sents_vec = top_vec[torch.arange(top_vec.size(0)).unsqueeze(1), clss]
#        sents_vec = sents_vec * mask_cls[:, :, None].float()
        sents_vec = top_vec.gather(1, clss.unsqueeze(-1).expand(-1, -1, 768))
        sents_vec = self.dropout(sents_vec)
#        cls_outs = sents_vec.clone().detach().cpu().numpy()
#        print(">>> ", cls_outs.shape, np.sum(cls_outs, axis=1)[:, :20])

        sent_scores, x, h = self.encoder(sents_vec, mask_cls)
        sent_scores = sent_scores.squeeze(-1)
        return sent_scores, mask_cls, x, h


In [ ]:
# setup tensorboard for visualizing results

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(log_dir=TBOARD_LOG_DIR)


## Training

In [ ]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    print('GPU device not found')


GPU device not found


In [ ]:
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

No GPU available, using the CPU instead.


In [ ]:
def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [ ]:
# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 16

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
test_dataloader = DataLoader(
            test_dataset, # The test samples.
            sampler = SequentialSampler(test_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )


# Create model

In [ ]:
model = Summarizer()
# tell pytorch to run on GPU
torch.cuda.empty_cache()
_= model.cuda()


In [ ]:
# Get all of the model's parameters as a list of tuples.
params = list(model.named_parameters())

print('The BERT model has {:} different named parameters.\n'.format(len(params)))

print('==== Embedding Layer ====\n')

for p in params[0:5]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== First Transformer ====\n')

for p in params[5:21]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))

print('\n==== Output Layer ====\n')

for p in params[-4:]:
    print("{:<55} {:>12}".format(p[0], str(tuple(p[1].size()))))




In [ ]:
## set all layers to train
for param in model.bert.parameters():
    param.requires_grad=True
for param in model.encoder.parameters():
    param.requires_grad=True

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, betas=(0.9, 0.999), eps=1e-08, weight_decay=0, amsgrad=False)


In [ ]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 2

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = int(total_steps * .2),
                                            num_training_steps = total_steps)


# Load checkpoint if required

In [ ]:
#load checkpoints if provided
cp_file = CP_DIR+'20201128.16.0.pth.tar'
if cp_file:
    m_state, o_state, s_state = load_cp(cp_file)
    model.load_cp(m_state)
    optimizer.load_state_dict(o_state)
    scheduler.load_state_dict(s_state)

This function writes out the labeled data as well as the result for evaluation using rouge

In [ ]:
def _get_ngrams(n, text):
    ngram_set = set()
    text_length = len(text)
    max_index_ngram_start = text_length - n
    for i in range(max_index_ngram_start + 1):
        ngram_set.add(tuple(text[i:i + n]))
    return ngram_set

def _block_tri(c, p):
    tri_c = _get_ngrams(3, c.split())
    for s in p:        
        tri_s = _get_ngrams(3, s.split())
        if len(tri_c.intersection(tri_s))>0:
            return True
    return False

In [ ]:
def gen_outputs(epoch, probs, labels, cls_ids, mask_cls,src,  meetings, is_test=False,):
    # extract sentences and labels

    f_handles_p = OrderedDict()
    f_handles_l = OrderedDict()
    f_content_p = OrderedDict()

    for i, m in enumerate(meetings):
        if m not in f_handles_p:
            if not is_test:
                f_handles_p[m] = open(RESULT_DIR+"PRED_"+str(m)+"_"+str(epoch)+".txt", "a")
                f_handles_l[m] = open(RESULT_DIR+"LABEL_"+str(m)+"_"+str(epoch)+".txt", "a")
            else:
                f_handles_p[m] = open(RESULT_DIR+"TPRED_"+str(m)+"_"+".txt", "a")
                f_handles_l[m] = open(RESULT_DIR+"TLABEL_"+str(m)+"_"+".txt", "a")
            p_file = RESULT_DIR+"PRED_"+str(m)+"_"+str(epoch)+".txt"
            if os.path.isfile(p_file):
                with open (RESULT_DIR+"PRED_"+str(m)+"_"+str(epoch)+".txt", "r") as pf:
                    f_content_p[m] = pf.readlines()
            else:
                f_content_p[m] = []

    sel_num = np.rint(np.sum(mask_cls, axis=1) * 0.15)
    selected_ids = np.argsort(-probs, 1)
    selected = dict()
    for y in range(probs.shape[0]):
        selected[y] = set()
        for x in range(probs.shape[1]):
            if x < sel_num[y]:
                selected[y].add(selected_ids[y,x])
    files = None


    f_names = [(a.name, b.name) for a, b in zip(f_handles_p.values(), f_handles_l.values())]
    try:
        for pi, passage in enumerate(src):
            #print(probs[p,:20])
            lines = tokenizer.decode(passage)
            lines = lines.split("[SEP]")
            for i, sent in enumerate(lines):
                sent = sent.replace("[SEP]", "").replace("[CLS]", "").replace("[PAD]", "")
                if labels[pi, i] == 1:
                    f_handles_l[meetings[pi]].write(sent + "\n")
                #if probs_bin[p, i] == 1:
                if i in selected[pi]:
                    if (len(sent.split()) >= 3 and not _block_tri(sent, f_content_p[meetings[pi]])):
                        f_content_p[meetings[pi]].append(sent)
                        #print(">>>",probs[p,i])
                        f_handles_p[meetings[pi]].write(sent + "\n")
    finally:
        for handle in list(f_handles_p.values()) + list(f_handles_l.values()):
            handle.close()
    #logger.debug(f_names)
    return f_names


In [ ]:
def avg_rouge(filesets):
  scores = list()  
  for i, (hyp_file, lab_file) in enumerate(filesets):
    with open(hyp_file, "r") as ref, open(lab_file, "r") as lab:
      reference = lab.read()
      hypothesis = ref.read()
      rouge = Rouge()
      try:
        score = rouge.get_scores(hypothesis, reference)
        scores.append(score)
      except:
        print("empty file", hyp_file, lab_file)
        pass

  # average rouge scores
  result_rouge = None
  for rouge_res in scores:
    if result_rouge is None:
      result_rouge = rouge_res[0]
      continue
    for key, obj in rouge_res[0].items():
      for k1, v1 in obj.items():
        result_rouge[key][k1] += v1
  results = None
  if len(scores) > 0:
    print("Averaging number of scores = ", len(scores))
    res = scores[0][0] # just for initializing keys. Values do not matter here
    for k, v in result_rouge.items():
        for k1, v1 in v.items():
            print(k,k1)
            res[k][k1] = v1/i
    results = res
  return results

In [ ]:
## setup loss function and other variables required for training

#seed_val = 42

#random.seed(seed_val)
#np.random.seed(seed_val)
#torch.manual_seed(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()
loss_c = torch.nn.BCELoss(reduction='none')


In [ ]:
# setup tensorboard for visualizing results

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(log_dir=TBOARD_LOG_DIR)


## Training and Validation

In [ ]:

# For each epoch...
for epoch_i in range(1, epochs):
    if EVAL:
        break
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode.
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):
        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            logger.debug('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))
        
        idx, src, labels, segs, clss, attn, mask_cls= batch
        idx, src, labels, segs, clss, attn, mask_cls = idx, src.to(device), labels.to(device), segs.to(device), clss.to(device), attn.to(device), mask_cls.to(device)

        # Always clear any previously calculated gradients before performing a
        # backward pass. PyTorch doesn't do this automatically because 
        # accumulating the gradients is "convenient while training RNNs". 
        # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
        model.zero_grad()        

        probs, mask_cls, x, h = model( src, segs, clss, attn, mask_cls)
        loss = loss_c(probs, labels.float())
        loss = (loss * mask_cls.float()).sum()

        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        total_train_loss += loss.item()

        # Perform a backward pass to calculate the gradients.
        loss.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()


    # save checkpoint
    #if (epoch_i+1 )%5 == 0:
    if True:
        logger.debug("saving checkpoint"+ str(epoch_i))
        save_cp(model.state_dict(), optimizer.state_dict(), scheduler.state_dict(), epoch_i)

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)
    writer.add_scalar("Loss/train", avg_train_loss, epoch_i)
    writer.flush()

    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)
    logger.debug("  Average training loss: {0:.2f}".format(avg_train_loss))
    logger.debug("  Training epcoh took: {:}".format(training_time))


    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.


    print("")
    print("Running Validation...")
    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    step = -1
    filesets = list()
    for batch in validation_dataloader:
        step += 1
        idx, src, labels, segs, clss, attn, mask_cls= batch
        idx, src, labels, segs, clss, attn, mask_cls = idx, src.to(device), labels.to(device), segs.to(device), clss.to(device), attn.to(device), mask_cls.to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        
            probs, mask_cls, x, h = model( src, segs, clss, attn, mask_cls)
            loss = loss_c(probs, labels.float())
            loss = (loss * mask_cls.float()).sum()
            # write results so that we can use rouge to compare
            if False:
                files = gen_outputs(epoch_i, probs.to('cpu').numpy(), labels.to('cpu').numpy(), clss.to('cpu').numpy(), mask_cls.to('cpu').numpy(), src.to('cpu').numpy(), idx.to('cpu').numpy())
                filesets.extend(files)

        # Accumulate the validation loss.
        total_eval_loss += loss.item()

    # Calculate the average loss over all of the batches.
    avg_eval_loss = total_eval_loss / len(validation_dataloader)
    logger.debug("  Average validation loss: {0:.2f}".format(avg_eval_loss))
    logger.debug("  Validation epcoh took: {:}".format(format_time(time.time() - t0)))

    writer.add_scalar("Loss/validation", avg_eval_loss, epoch_i)
    writer.flush()

    #rouge scores for the epoch
    #if (epoch_i+1 )%5 == 0:
    if False:
        logger.debug("prior to running rouge")
        rouge_scores = avg_rouge(filesets)
        logger.debug("completed running rouge")
        logger.debug(rouge_scores)
        if rouge_scores is not None:
            for r_type in ("rouge-1", 'rouge-2'):
                for metric in ("f", "p", "r"):
                    writer.add_scalar(r_type+"/"+metric+"/"+"Validation", rouge_scores[r_type][metric], epoch_i)
        writer.flush()

print("")
print("Training complete!")
writer.close()
print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))


In [ ]:
!kill 1799

# Testing

In [ ]:
# ========================================
#               Testing
# ========================================


print("")
print("Running Test...")
t0 = time.time()

# Put the model in evaluation mode--the dropout layers behave differently
# during evaluation.
model.eval()

# Tracking variables 
total_test_accuracy = 0
total_test_loss = 0
nb_eval_steps = 0

# Evaluate data for one epoch
step = -1
filesets = list()
for batch in test_dataloader:
    step += 1
    idx, src, labels, segs, clss, attn, mask_cls= batch
    idx, src, labels, segs, clss, attn, mask_cls = idx, src.to(device), labels.to(device), segs.to(device), clss.to(device), attn.to(device), mask_cls.to(device)
    
    # Tell pytorch not to bother with constructing the compute graph during
    # the forward pass, since this is only needed for backprop (training).
    with torch.no_grad():        
        probs, mask_cls, x, h = model( src, segs, clss, attn, mask_cls)
        loss = loss_c(probs, labels.float())
        loss = (loss * mask_cls.float()).sum()
        # write results so that we can use rouge to compare
        files = gen_outputs(None, probs.to('cpu').numpy(), labels.to('cpu').numpy(), clss.to('cpu').numpy(), mask_cls.to('cpu').numpy(), src.to('cpu').numpy(), idx.to('cpu').numpy(), is_test=True)
        filesets.extend(files)
        logger.debug("TEST LOSS TOTAL"+str(loss.item()))

    # Accumulate the test loss.
    total_test_loss += loss.item()

# Calculate the average loss over all of the batches.
avg_test_loss = total_test_loss / len(test_dataloader)
logger.debug("  Average Test loss: {0:.2f}".format(avg_test_loss))

writer.add_scalar("Loss/Test", avg_test_loss)
writer.flush()

#rouge scores for the epoch
rouge_scores = avg_rouge(filesets)
logger.debug(rouge_scores)
if rouge_scores is not None:
    for r_type in ("rouge-1", 'rouge-2'):
        for metric in ("f", "p", "r"):
            print("writing metrics")
            writer.add_scalar(r_type+"/"+metric+"/"+"Test", rouge_scores[r_type][metric])
writer.flush() 

In [ ]:
print(TBOARD_LOG_DIR)

In [ ]:
%load_ext tensorboard

%tensorboard --logdir="/content/drive/My Drive/summ_data/cnn/tboard/"